In [4]:
# 환경설정
from dotenv import load_dotenv
load_dotenv()

True

# 1. Chain을 이용한 Simple LLM
1. PromptTemplate
2. LLM
3. OutputParser

In [5]:
from langchain_openai import ChatOpenAI
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate

In [6]:
# 1. PromptTemplate
# from langchain import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate

# prompt = PromptTemplate(
#     template='가장 강한 포켓몬은?',
#     input_variables=[]
# )

prompt = ChatPromptTemplate([
    ('system', '너는 애니메이션을 잘 아는 챗봇이야. 사용자의 질문에 친절하고 상세하게 답할 수 있어.'),
    ('user', '{question}')
])



In [7]:
# 2. LLM
model = ChatOpenAI(
    model_name='gpt-4o-mini',
    temperature=0.3
)

In [8]:
# 3. OutputParser (StrOutputParser)
from langchain_core.output_parsers import StrOutputParser
output = StrOutputParser()

In [9]:
# # Chain -> 질의
# # question: 어떤 포켓몬이 가장 강해?

# chain = prompt | model | output

# result = chain.invoke({'question': '어떤 포켓몬이 가장 강해?'})



In [10]:
# print(result)

In [11]:
# from langchain_core.runnables import RunnableSequence
# chain = RunnableSequence(prompt, model, output)
# chain.invoke({'question': '어떤 포켓몬이 가장 강해?'})

### 단계별 Chatbot
- 첫 대화에서 내 이름을 알려주고, 다음 대화에서 내 이름 기억하는지 물어보기!

1. 그냥 Chat
- langchain_openai의 ChatOpenAI
- langchain_core.messages의 클래스

In [12]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

model = ChatOpenAI(model_name='gpt-4o-mini', temperature=0.5)


In [13]:
message = HumanMessage(content="안녕 내 이름은 sj야")
response = model.invoke([message])

print(f"사용자: 안녕 내 이름은 sj야")
print(f"AI: {response.content}")

message2 = HumanMessage(content="내 생일이 언제라고?")
response2 = model.invoke([message2])

print(f"사용자: 내 생일이 언제라고?")
print(f"AI: {response2.content}")

사용자: 안녕 내 이름은 sj야
AI: 안녕하세요, sj님! 만나서 반갑습니다. 어떻게 도와드릴까요?
사용자: 내 생일이 언제라고?
AI: 죄송하지만, 당신의 생일에 대한 정보는 알 수 없습니다. 생일을 말씀해 주시면 축하해 드릴 수 있습니다!


2. 직접 대화 맥락 유지
- langchain_openai의 ChatOpenAI
- langchain_core.messages의 클래스

In [14]:
messages = [
    SystemMessage(content="너는 대화를 잘하는 AI 어시스턴트야.")
]

# 첫 번째 메시지
messages.append(HumanMessage(content="안녕 내 이름은 sj야"))
response1 = model.invoke(messages)
messages.append(response1)

print(f"사용자: 안녕 내 이름은 sj야")
print(f"AI: {response1.content}")

# 두 번째 메시지 (맥락 유지)
messages.append(HumanMessage(content="내 생일이 언제라고?"))
response2 = model.invoke(messages)

print(f"사용자: 내 생일이 언제라고?")
print(f"AI: {response2.content}")

사용자: 안녕 내 이름은 sj야
AI: 안녕하세요, sj님! 만나서 반갑습니다. 어떻게 도와드릴까요?
사용자: 내 생일이 언제라고?
AI: 죄송하지만, sj님의 생일에 대한 정보는 알 수 없어요. 하지만 생일이 언제인지 말씀해 주시면, 축하 메시지를 준비해 드릴 수 있어요!


3. Memory로 대화 맥락 유지
- langchain_openai의 ChatOpenAI
- langchain_core.messages의 클래스
- langchain_core.runnables의 클래스
- langchain_core.prompts의 클래스

In [15]:
from langchain_core.chat_history import BaseChatMessageHistory

class InMemoryHistory(BaseChatMessageHistory):
    def __init__(self):
        self.messages = []

    def add_messages(self, messages):
        self.messages.extend(messages)

    def clear(self):
        self.messages = []

    def __repr__(self):
        return str(self.messages)

In [16]:
store = {}  
def get_by_session_id(session_id):
    if session_id not in store:
        store[session_id] = InMemoryHistory()
    return store[session_id]

In [17]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

prompt = ChatPromptTemplate.from_messages([
    ('system', '너는 {skill}을 잘하는 AI 어시스턴트야.'),
    MessagesPlaceholder(variable_name='history'),
    ('human', '{query}')    
])
model = ChatOpenAI(model_name='gpt-4o-mini', temperature=0.5)
chain = prompt | model

chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history=get_by_session_id,
    input_messages_key='query',
    history_messages_key='history'    
)

res = chain_with_history.invoke(
    {'skill': '대화', 'query': '안녕 내 이름은 sj야'},
    config={
        'configurable': {'session_id': 'squirrel'}
    }
)

res2 = chain_with_history.invoke(
    {'skill': '대화', 'query': '내 생일이 언제라고?'},
    config={
        'configurable': {'session_id': 'squirrel'}
    }
)

print(res.content)
print(res2.content)

안녕, sj! 만나서 반가워. 어떻게 도와줄까요?
죄송하지만, sj님의 생일에 대한 정보는 알 수 없어요. 생일이 언제인지 말씀해주시면 축하해드릴 수 있어요!
